In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from utils.sersic_functions import sersic_profile, generate_random_pos
from utils.download.h2f_download_functions import get_galaxy_img

In [ ]:
df = pd.read_csv("..\data\SERSIC\df_train.csv")

### EDA

In [ ]:
# Radio de las elipses de Sersic (3 veces el radio) en pixeles
# En este caso seleccionamos 210 pixeles de radio

print(df["rSerRadius"].max()*3/0.25)
print(df["rSerRadius"].median()*3/0.25)
print(df["rSerRadius"].mean()*3/0.25)
print(df["rSerRadius"].std()*3/0.25)

data = df["rSerRadius"] * 3 / 0.25

# Definir bines logarítmicos desde 1 a 1000 (puedes ajustar la cantidad de bines)
bins = np.logspace(np.log10(1), np.log10(1000), num=20)

plt.hist(data, bins=np.linspace(0,300,30), histtype="step", linewidth=1.5)


#plt.xscale("log")
plt.yscale("log")
plt.xlabel("Major Axis [pix]")
plt.ylabel("Frecuencia")
plt.show()

### Funciones

In [ ]:
def plot_galaxy_and_sersic(df, idx, Ie=1.0, n=4):
    """
    Visualiza una galaxia con su perfil radial, su perfil de Sérsic y distribuciones de posiciones 
    aleatorias de supernovas simuladas.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame con parámetros de galaxias y supernovas. Debe contener 
        las columnas: 
        - 'host_ra', 'host_dec' (coordenadas en grados, usadas en `get_sdss_img`)
        - 'rSerRadius' (radio efectivo en arcsec)
        - 'rSerAb' (relación de ejes b/a)
        - 'rSerPhi' (ángulo de posición en grados)
        - 'dx', 'dy' (desplazamiento en píxeles de la supernova respecto al centro).
    idx : int
        Índice de la fila en el DataFrame correspondiente a la galaxia a visualizar.
    Ie : float, optional
        Intensidad central del perfil de Sérsic. Default es 1.0.
    n : int, optional
        Índice de Sérsic (concentración del perfil). Default es 4 (perfil de de Vaucouleurs).

    Returns
    -------
    sersic_img : np.ndarray
        Imagen 2D con el perfil de Sérsic generado para la galaxia.
    random_pos : np.ndarray
        Arreglo de forma (1000, 2) con posiciones aleatorias simuladas de supernovas 
        dentro del perfil de Sérsic, centradas en el núcleo de la galaxia.

    Notes
    -----
    - La primera figura muestra la galaxia real con el perfil elíptico y 
      la posición de la supernova observada.
    - La segunda figura muestra el perfil de Sérsic simulado.
    - La tercera figura muestra un mapa de calor con las posiciones simuladas 
      de la supernova en 1000 iteraciones.
    """
    row = df.iloc[idx]
    image_data = get_galaxy_img(df, idx, level=0, size=240) #Imagen de 240x240 con resolucion 0.25 arcsec/pix

    # Coordenadas del centro
    x_center, y_center = image_data.shape[0]//2 -1 , image_data.shape[1]//2 -1

    # Parámetros de Sérsic
    Re_arcsec = row["rSerRadius"]
    b_over_a = row["rSerAb"]
    theta_sersic = row["rSerPhi"]  # Usar el ángulo original
    pixel_scale = 0.25
    Re_pix = Re_arcsec / pixel_scale
    A_pix = Re_pix
    B_pix = A_pix * b_over_a

    # Generar perfil de Sérsic
    sersic_img = sersic_profile(
        image_shape=image_data.shape,
        x_center=x_center, y_center=y_center,
        Re_arcsec=Re_arcsec,
        b_over_a=b_over_a,
        theta_deg=theta_sersic,  
        pixel_scale=pixel_scale,
        Ie=Ie,
        n=n
    )

    # vemos las posiciones que tomo la supernova en 1000 iteraciones
    random_pos = []
    for i in range(1000):
        pos = generate_random_pos(sersic_radius = Re_arcsec, 
                                    sersic_ab = b_over_a, 
                                    sersic_phi = theta_sersic, 
                                    img_size = image_data.shape[0])
        random_pos.append(pos) 

    random_pos =  np.array(random_pos) + x_center
    x_iter = random_pos[:, 0]
    y_iter = random_pos[:, 1]

    fig, axes = plt.subplots(1, 3, figsize=(18, 5), dpi=300)

    dx, dy = row[["dx","dy"]].values

    x_star = x_center + dx
    y_star = y_center + dy

    # Imagen real con elipse
    ax1 = axes[0]

    # Posicion SN real
    ax1.scatter(x_star, y_star,
                marker='*',         
                color='yellow',     
                s=500,              
                edgecolor='black')  
    
    ax1.imshow(image_data, cmap='gray', origin="lower",
               vmin=np.percentile(image_data, 1),
               vmax=np.percentile(image_data, 99))
    

    for r in [1, 2, 3]:
        ellipse = Ellipse((x_center, y_center),
                        width=B_pix * 2 * r,
                        height=A_pix * 2 * r,
                        angle=theta_sersic+90,
                        edgecolor='red',
                        facecolor='none',
                        lw=1.5,
                        linestyle='--' if r < 3 else '-')
        ax1.add_patch(ellipse)

    ax1.set_title("Galaxia con perfil elíptico")
    ax1.axis('off')

    # Mostrar perfil de Sérsic
    ax2 = axes[1]
    im = ax2.imshow(sersic_img, cmap='viridis', vmin=0, vmax=300, origin="lower")

    # Posciones SN arbitrarias
    for r in [1, 2, 3]:
        ellipse = Ellipse((x_center, y_center),
                        width=B_pix * 2 * r,
                        height=A_pix * 2 * r,
                        angle=theta_sersic+90,
                        edgecolor='red',
                        facecolor='none',
                        lw=1.5,
                        linestyle='--' if r < 3 else '-')
        ax2.add_patch(ellipse)

    ax2.set_title("Perfil de Sérsic (n=4)")
    ax2.axis('off')
    fig.colorbar(im, ax=ax2, label='Intensidad')

    ax3 = axes[2]
    hist, _, _ = np.histogram2d(x_iter, y_iter, bins=240, range=[[0, 240], [0, 240]])
    im3 = ax3.imshow(hist.T, cmap='hot', interpolation='nearest', origin="lower")
    ax3.set_title('Posiciones supernova 1000 sampleos arbitrarios')
    ax3.axis('off')
    fig.colorbar(im3, ax=ax3, label='Frecuencia')


    plt.tight_layout()
    plt.show()

    return sersic_img, random_pos

### Plots

In [ ]:
for idx in range(7):
    _ = plot_galaxy_and_sersic(df = df,
                               idx = idx) 